In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ensure plots show up inside Jupyter Notebook (if you're using one)
#%matplotlib inline  

# Load data (adjust the path to your CSV file)
# If your file has no header, set header=None
df = pd.read_csv(r"C:\Users\jay.jain\Desktop\Codes - Jay\Assignment 1\YBAc9.csv", header=None)


# Rename columns based on your format
df.columns = [    "RIC", "Alias", "Type", "DateTime", "GMT Offset", "Interval",
    "Open", "High", "Low", "Close", "Volume"]

# Convert datetime column
df["DateTime"] = pd.to_datetime(df["DateTime"], errors="coerce")

# Drop rows with invalid or missing DateTime
df.dropna(subset=["DateTime"], inplace=True)

# ✅ Filter for 2012–2015 only
df = df[(df["DateTime"] >= "2023-01-01") & (df["DateTime"] <= "2024-12-31")]

# Ensure numeric columns are in correct dtype
for col in ["Open", "High", "Low", "Close", "Volume"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing data
df.dropna(subset=["Close", "Volume"], inplace=True)

# Set price value for volume profile — Close or OHLC average
df["Price"] = df["Close"]
# Optional: use average price instead
# df["Price"] = (df["Open"] + df["High"] + df["Low"] + df["Close"]) / 4

# Define bin width (adjust based on price range granularity)
bin_width = 0.01

# Create bins
min_price = df["Price"].min()
max_price = df["Price"].max()
bins = np.arange(min_price, max_price + bin_width, bin_width)

# Bin prices
df["PriceBin"] = pd.cut(df["Price"], bins)

# Aggregate volume per price bin
volume_profile = df.groupby("PriceBin")["Volume"].sum()

# Get bin centers for plotting
bin_centers = [interval.mid for interval in volume_profile.index]

# Plot the volume profile (horizontal bar plot)
plt.figure(figsize=(10, 8))
plt.barh(bin_centers, volume_profile.values, height=bin_width, color="cornflowerblue")
plt.xlabel("Volume")
plt.ylabel("Price")
plt.title("Composite Volume Profile")
plt.grid(True)
plt.tight_layout()
plt.show()